In [246]:
# Data Manipulation libraries
import numpy as np
import pandas as pd

# Sci-Kit Learn Processing and Evaluating
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import KFold, cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler, QuantileTransformer


# Supervised Learning Models  
from sklearn.dummy import DummyRegressor
from sklearn.linear_model import LinearRegression 
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso  
from sklearn.linear_model import ElasticNet
from sklearn.tree import DecisionTreeRegressor  
from sklearn.neighbors import KNeighborsRegressor 
from sklearn.svm import SVR  
from sklearn.ensemble import RandomForestRegressor  
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.neural_network import MLPRegressor

# Visualization libraries
import matplotlib.pyplot as plt
import seaborn as sns
import altair as alt

# Let's set our Random State here as well
random_state = 9

In [247]:
root_path = '../../datasets/'
X_train_file = 'X_train_filled_KPIs_QoQ_PCA.csv'
y_train_file = 'y_train.csv'
X_old = 'train_1003.csv'
X = pd.read_csv(root_path+X_train_file)
y = pd.read_csv(root_path+y_train_file)
old = pd.read_csv(root_path+X_old)
print(X.shape, y.shape, old.shape)



(1910, 326) (1974, 19) (1450, 287)


In [248]:
X.head()


,Ticker,Name,Sector,CapitalExpenditure_2024Q2,CapitalExpenditure_2024Q3,CapitalExpenditure_2024Q4,CapitalExpenditure_2025Q1,CashAndSTInvestments_2024Q2,CashAndSTInvestments_2024Q3,CashAndSTInvestments_2024Q4,...,PCA_KPI_PC2,PCA_KPI_PC3,PCA_KPI_PC4,PCA_KPI_PC5,PCA_KPI_PC6,PCA_KPI_PC7,PCA_KPI_PC8,PCA_KPI_PC9,PCA_KPI_PC10,Cluster
0,ACIW,ACI WORLDWIDE INC,Information Technology,-3777000.0,-4045500.0,-4663500.0,-4112000.0,1.851080e+08,1.970750e+08,188364000.0,...,-0.123215,0.554197,0.635466,1.822318,0.574667,-0.658662,0.246484,0.793617,0.242835,0
1,HONE,HARBORONE BANCORP INC,Financials,-220000.0,-569000.0,-339000.0,-208000.0,2.350620e+08,2.242790e+08,231071000.0,...,1.653147,-1.471920,-0.344522,1.129452,-1.540170,0.720196,-0.793347,0.415490,-0.310096,1
2,REPL,REPLIMUNE GROUP INC,Health Care,-1618000.0,-1114000.0,-2266000.0,-1503000.0,8.278500e+07,7.524700e+07,78303500.0,...,-0.800499,0.397648,-0.933713,0.954683,-0.089639,0.738622,-0.598111,-0.966618,0.457380,6
3,RBRK,RUBRIK INC CLASS A,Information Technology,-15766000.0,-4929000.0,-7527000.0,-8401000.0,4.293020e+08,1.423490e+08,103896000.0,...,-1.510561,-0.243133,-2.950360,-0.545931,0.209082,0.175673,0.821812,-0.060885,0.232810,6
4,CSL,CARLISLE COMPANIES INC,Industrials,-24900000.0,-19300000.0,-36600000.0,-29000000.0,1.736300e+09,1.530600e+09,753500000.0,...,-0.909726,0.196795,-0.543470,-1.068878,-0.480741,-1.400439,-0.783270,0.340897,-1.379224,3


In [249]:
y.head()

,Ticker,CapitalExpenditure_2025Q2,CashAndSTInvestments_2025Q2,CashFromOps_2025Q2,CostOfRevenue_2025Q2,CurrentAssets_2025Q2,CurrentLiabilities_2025Q2,EPS_2025Q2,IncomeTaxExpense_2025Q2,InterestExpense_2025Q2,LongTermDebt_2025Q2,NetIncome_2025Q2,OperatingIncome_2025Q2,OtherOperatingExpense_2025Q2,Revenue_2025Q2,TotalAssets_2025Q2,TotalDebt_2025Q2,TotalEquity_2025Q2,TotalLiabilities_2025Q2
0,ACIW,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,HONE,-139000.0,203053000.0,-4987000.0,44301000.0,NaN,NaN,0.20,2569000.0,32385000.0,439652000.0,8058000.0,NaN,32385000.0,44301000.0,5.609075e+09,439652000.0,580147000.0,5.028928e+09
2,REPL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,RBRK,-6315000.0,283998000.0,39655000.0,60483000.0,1.116895e+09,961289000.0,-0.53,1274000.0,9813000.0,322821000.0,-102104000.0,-93091000.0,311089000.0,278481000.0,1.474606e+09,334571000.0,-556530000.0,2.031136e+09
4,BRCC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [250]:
new = X.merge(y, on='Ticker')
print(old.shape,new.shape)

(1450, 287) (1910, 344)


In [251]:
old.set_index('Ticker',inplace=True)
new.set_index('Ticker',inplace=True)
old.sort_index(inplace=True)
new.sort_index(inplace=True)



In [252]:
old.head(1)

,ID,Name,Sector,CapitalExpenditure_2024Q2,CapitalExpenditure_2024Q3,CapitalExpenditure_2024Q4,CapitalExpenditure_2025Q1,CashAndSTInvestments_2024Q2,CashAndSTInvestments_2024Q3,CashAndSTInvestments_2024Q4,...,InterestExpense_Rate,CashFromOps_Rate,TotalAssets_Rate,TotalLiabilities_Rate,OperatingIncome_Rate,TotalDebt_Rate,KPI_NetProfitMargin_Rate,NetIncome_2025Q2,Revenue_2025Q2,KPI_NetProfitMargin_2025Q2
Ticker,,,,,,,,,,,,,,,,,,,,,
A,262,AGILENT TECHNOLOGIES INC,Health Care,-39486000,-92000000.0,-93000000.0,-97000000,796798000.0,1.779000e+09,1.329000e+09,...,1984900.0,82214900.0,992170000.0,661200000.0,66848100.0,309299600.0,0.030873,215000000,1.668000e+09,0.128897


In [253]:
new.head(1)

,Name,Sector,CapitalExpenditure_2024Q2,CapitalExpenditure_2024Q3,CapitalExpenditure_2024Q4,CapitalExpenditure_2025Q1,CashAndSTInvestments_2024Q2,CashAndSTInvestments_2024Q3,CashAndSTInvestments_2024Q4,CashAndSTInvestments_2025Q1,...,InterestExpense_2025Q2,LongTermDebt_2025Q2,NetIncome_2025Q2,OperatingIncome_2025Q2,OtherOperatingExpense_2025Q2,Revenue_2025Q2,TotalAssets_2025Q2,TotalDebt_2025Q2,TotalEquity_2025Q2,TotalLiabilities_2025Q2
Ticker,,,,,,,,,,,,,,,,,,,,,
A,AGILENT TECHNOLOGIES INC,Health Care,-39486000.0,-92000000.0,-93000000.0,-97000000.0,796798000.0,1.779000e+09,1.329000e+09,1.467000e+09,...,29000000.0,3.349000e+09,215000000.0,300000000.0,566000000.0,1.668000e+09,1.215800e+10,3.495000e+09,6.136000e+09,6.022000e+09


In [254]:
old.sample(1)

,ID,Name,Sector,CapitalExpenditure_2024Q2,CapitalExpenditure_2024Q3,CapitalExpenditure_2024Q4,CapitalExpenditure_2025Q1,CashAndSTInvestments_2024Q2,CashAndSTInvestments_2024Q3,CashAndSTInvestments_2024Q4,...,InterestExpense_Rate,CashFromOps_Rate,TotalAssets_Rate,TotalLiabilities_Rate,OperatingIncome_Rate,TotalDebt_Rate,KPI_NetProfitMargin_Rate,NetIncome_2025Q2,Revenue_2025Q2,KPI_NetProfitMargin_2025Q2
Ticker,,,,,,,,,,,,,,,,,,,,,
HOPE,1644,HOPE BANCORP INC,Financials,-2533000,-2900000.0,-2267000.0,-2659000,654044000.0,680857000.0,458199000.0,...,-3696500.0,14332400.0,-122050600.0,-133147900.0,-5415400.0,-8915800.0,-0.010423,-27881000,94577000.0,-0.294797


In [255]:
compare1 = old.loc['SSNC','KPI_NetProfitMargin_Rate']
compare2 = new.loc['SSNC','KPI_NetProfitMargin_Rate']
print(compare1,'/',compare2)

0.007886824 / 0.0078868240160804


In [256]:
old.isna().sum().max()

0

In [257]:
new.isna().sum().max()
#new.dropna(inplace=True)

691

In [258]:
print(old.shape,new.shape)

(1450, 286) (1910, 343)


In [259]:
old['Market Cap'].unique()

array(['Mid-Cap', 'Small-Cap', 'Mega-Cap', 'Large-Cap', 'Micro-Cap'],
      dtype=object)

In [260]:
new['Market Cap'].unique()

array(['Mid-Cap', 'Small-Cap', 'Mega-Cap', 'Large-Cap', 'Micro-Cap'],
      dtype=object)

In [261]:
len(new['Name'].unique())

1909

In [262]:
len(old[old['Market Cap']=='Mega-Cap'])

7

In [263]:
len(new[new['Market Cap'] =='Mega-Cap'])

7

In [264]:
old_tickers = list(old.index)
new_tickers = list(new.index)
in_old_not_new = []
for ticker in old_tickers:
    if ticker not in new_tickers:
        in_old_not_new.append(ticker)
print(len(in_old_not_new))

68


In [265]:
in_new_not_old = []
for ticker in new_tickers:
    if ticker not in old_tickers:
        in_new_not_old.append(ticker)
print(len(in_new_not_old))

528


In [266]:
# We need to look at the tickers that were in old
sample = new[new.index.isin(in_new_not_old)]
sample[['Sector','Exchange','Market Cap','Location',]].head()

,Sector,Exchange,Market Cap,Location
Ticker,,,,
AAP,Consumer Discretionary,New York Stock Exchange Inc.,Small-Cap,1
ABM,Industrials,New York Stock Exchange Inc.,Small-Cap,1
ABR,Financials,New York Stock Exchange Inc.,Small-Cap,1
ABSI,Health Care,NASDAQ,Micro-Cap,1
ACA,Industrials,New York Stock Exchange Inc.,Small-Cap,1


In [267]:
print(len(sample))

528


In [268]:
sample.iloc[:,-14:].isna().sum()

CurrentAssets_2025Q2            457
CurrentLiabilities_2025Q2       452
EPS_2025Q2                      452
IncomeTaxExpense_2025Q2         452
InterestExpense_2025Q2          450
LongTermDebt_2025Q2             447
NetIncome_2025Q2                452
OperatingIncome_2025Q2          466
OtherOperatingExpense_2025Q2    453
Revenue_2025Q2                  452
TotalAssets_2025Q2              443
TotalDebt_2025Q2                444
TotalEquity_2025Q2              443
TotalLiabilities_2025Q2         443
dtype: int64

In [269]:
#Let's see if these are in test
test_path = 'X_test_filled_KPIs_QoQ_PCA.csv'
test = pd.read_csv(root_path+test_path)
print(test.head())

  Ticker                   Name         Sector  CapitalExpenditure_2024Q2  \
0     SF  STIFEL FINANCIAL CORP     Financials                -33952000.0   
1     GH    GUARDANT HEALTH INC    Health Care                 -5077000.0   
2   TGLS         TECNOGLASS INC    Industrials                -20302000.0   
3   CARG   CARGURUS INC CLASS A  Communication                -31226000.0   
4    BKU         BANKUNITED INC     Financials                 51939000.0   

   CapitalExpenditure_2024Q3  CapitalExpenditure_2024Q4  \
0                -12769000.0                -17431000.0   
1                 -4199000.0                -18875000.0   
2                -23685000.0                -25690000.0   
3                -14895000.0                -13698000.0   
4                 -2547000.0                 -3407000.0   

   CapitalExpenditure_2025Q1  CashAndSTInvestments_2024Q2  \
0                -16573000.0                 2.615670e+09   
1                 -4459000.0                 1.035239e+09   

In [270]:
print(len(in_old_not_new),len(test[test['Ticker'].isin(in_old_not_new)]))

68 68


In [271]:
new = new.dropna(subset=['Revenue_2025Q2','NetIncome_2025Q2'])

In [272]:
new.shape

(1388, 343)

In [273]:
old_tickers = list(old.index)
new_tickers = list(new.index)
in_old_not_new = []
for ticker in old_tickers:
    if ticker not in new_tickers:
        in_old_not_new.append(ticker)
print(len(in_old_not_new))

138


In [274]:
in_new_not_old = []
for ticker in new_tickers:
    if ticker not in old_tickers:
        in_new_not_old.append(ticker)
print(len(in_new_not_old))

76


In [275]:
new = new[(new['Market Cap'] != 'Mega-Cap')]
new.shape

(1381, 343)